In [ ]:
import os
import shutil
import pandas as pd
import re
import string
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
wordnet_lemmatizer = WordNetLemmatizer()
#import spacy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import PIL
from PIL import Image
import urllib.request
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Input
from keras.layers import Conv2D, MaxPooling2D, concatenate
from keras.utils import to_categorical
from keras.preprocessing import image

from tqdm import tqdm
from keras.models import load_model, Model
from pathlib import Path


from sklearn.model_selection import cross_val_score


In [ ]:
from google.colab import drive
import zipfile
import os

# Mount Google Drive
drive.mount('/content/drive')

# Path to the zip file in Google Drive
zip_file_path = '/content/drive/My Drive/RMTC_IMPLEMENTATION/train_images.zip'

# Directory to extract the contents to
extract_to_dir = '/content/train_images'

# Create the directory if it doesn't exist
os.makedirs(extract_to_dir, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_dir)

# List the extracted files
extracted_files = os.listdir(extract_to_dir)
print("Files extracted to:", extract_to_dir)
print("Extracted files:", extracted_files)


Mounted at /content/drive
Files extracted to: /content/train_images
Extracted files: ['train_images']


In [ ]:
# Path to the zip file in Google Drive
zip_file_path = '/content/drive/My Drive/RMTC_IMPLEMENTATION/test_images.zip'

# Directory to extract the contents to
extract_to_dir = '/content/test_images'

# Create the directory if it doesn't exist
os.makedirs(extract_to_dir, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_dir)

# List the extracted files
extracted_files = os.listdir(extract_to_dir)
print("Files extracted to:", extract_to_dir)
print("Extracted files:", extracted_files)

Files extracted to: /content/test_images
Extracted files: ['test_images']


In [ ]:
from sklearn.base import BaseEstimator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

class CustomKerasRegressor(BaseEstimator):
    def __init__(self, epochs=10, batch_size=32):
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(512, activation='relu', input_shape=(input_dim,)),
            Dropout(0.5),
            Dense(input_dim, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy')
        return model

    def fit(self, X, y):
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size)

    def predict(self, X):
        return self.model.predict(X)


In [ ]:
pd.set_option('display.max_rows',None)
train=pd.read_csv(r'newtraindata.csv')
train=train.replace(np.nan, '',regex=True)
#val=pd.read_csv(r'.\Dataset\validation.csv')
#val=val.replace(np.nan, '',regex=True)
test=pd.read_csv(r'newtestdata.csv')
test=test.replace(np.nan, '',regex=True)

In [ ]:

punctuaions=set(string.punctuation)

#inputs
X_text_train=train['title'].map(str)+' '+train['description'].map(str)
#X_text_val=val['title'].map(str)+' '+val['description'].map(str)
X_text_test=test['title'].map(str)+' '+test['description'].map(str)
#outputs
y_text_train=train['categories__2'].values
y_text_test=test['categories__2'].values
#y_text_val=val['categories__2'].values
#y_text_val=[x.lower() for x in y_text_val ]
y_text_train=[x.lower() for x in y_text_train ]
y_text_test=[x.lower() for x in y_text_test ]


In [ ]:
unique_leaf_node =set( leafnode_extraction(train)+ leafnode_extraction(test))
print(len(unique_leaf_node))

329


In [ ]:
def leafnode_extraction(data_frame):
    leafnode=[]
    for index,col in data_frame.iterrows():
        if len(col['categories__6'])!=0:
            leafnode.append(col['categories__6'])
        elif len(col['categories__5'])!=0:
            leafnode.append(col['categories__5'])
        elif len(col['categories__4'])!=0:
            leafnode.append(col['categories__4'])
        elif len(col['categories__3'])!=0:
            leafnode.append(col['categories__3'])
        else:
            leafnode.append(col['categories__2'])
    return leafnode



#feature extraction
def tf_idf_vectorization(inp,test):
    X_train_tfidf=[]
    X_new_tfidf=[]
    #X_val_tfidf=[]
    tfidf_vect1=TfidfVectorizer(max_features=5000)
    tfidf_vect1.fit(inp['final_lemmatized_string'])
    X_train_tfidf = tfidf_vect1.transform(inp['final_lemmatized_string'])
    X_new_tfidf = tfidf_vect1.transform(test['final_lemmatized_string'])
    #X_val_tfidf = tfidf_vect1.transform(val['final_lemmatized_string'])
    print(X_train_tfidf.shape)
    print(X_new_tfidf.shape)
    return  X_train_tfidf,X_new_tfidf
def create_model(dims,n):
    model=Sequential()
    model.add(Dense(512, input_dim=dims[1],activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(n, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
    return model


In [ ]:
#stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}','&','nan'])
stemmer = SnowballStemmer('english')
#processed_train = []


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Preprocess the text in training and testing dataset stemming and lemmatizing
nltk.download('wordnet')
lemmatized_train =[]
lemmatized_test=[]
lemmatized_val=[]

for doc in X_text_train:
    tokens = word_tokenize(doc)
    tokens=[w.lower() for w in tokens if w.isalpha()]
    filtered = [word for word in tokens if word not in stop_words]
    lemmatized=[wordnet_lemmatizer.lemmatize(word) for word in filtered]
    lemmatized_train.append(lemmatized)

for doc in X_text_test:
    tokens = word_tokenize(doc)
    tokens=[w.lower() for w in tokens if w.isalpha()]
    filtered = [word for word in tokens if word not in stop_words]
    lemmatized=[wordnet_lemmatizer.lemmatize(word) for word in filtered]
    lemmatized_test.append(lemmatized)


#for doc in X_text_val:
    #tokens = word_tokenize(doc)
    #tokens=[w.lower() for w in tokens if w.isalpha()]
    #filtered = [word for word in tokens if word not in stop_words]
    #lemmatized=[wordnet_lemmatizer.lemmatize(word) for word in filtered]
    #lemmatized_val.append(lemmatized)
#print(lemmatized_train[1])
train['lemmatized_text']=lemmatized_train
test['lemmatized_text']=lemmatized_test
#val['lemmatized_text']=lemmatized_val


# Converting array to string
row_lst1 =[]
for lst in train.loc[:,'lemmatized_text']:
    text = ''
    for word in lst:
        text = text + ' ' + word
    row_lst1.append(text)
train['final_lemmatized_string']=row_lst1
#train.drop('final_stemmed_text',axis=1,inplace=True)
train.head()
row_lst1 =[]

for lst in test.loc[:,'lemmatized_text']:
    text = ''
    for word in lst:
        text = text + ' ' + word
    row_lst1.append(text)

test['final_lemmatized_string']=row_lst1
test.head()
#row_lst1 =[]
#for lst in val.loc[:,'lemmatized_text']:
    #text = ''
    #for word in lst:
        #text = text + ' ' + word
    #row_lst1.append(text)
#val['final_lemmatized_string']=row_lst1
#val.head()



[nltk_data] Downloading package wordnet to /root/nltk_data...


,title,description,imUrl,price,brand,categories__1,categories__2,categories__3,categories__4,categories__5,categories__6,lemmatized_text,final_lemmatized_string
0,Panasonic SC-AK29 Compact Stereo System (Disco...,"Equipped with a five-CD changer, Panasonic's S...",http://ecx.images-amazon.com/images/I/41QGRQSQ...,134.50,,Electronics,Home Audio,Compact Stereos,,,,"[panasonic, compact, stereo, system, discontin...",panasonic compact stereo system discontinued ...
1,Sennheiser RS4-9 On-Ear Wireless Headphone,Sennheiser's RS4-9 wireless headphones consist...,http://g-ecx.images-amazon.com/images/G/01/x-s...,134.50,,Electronics,Accessories & Supplies,Audio & Video Accessories,Headphones,,,"[sennheiser, wireless, headphone, sennheiser, ...",sennheiser wireless headphone sennheiser wire...
2,Pioneer VSX-D309 Audio/Video Receiver (Discont...,"With built-in Dolby Digital, DTS, and Dolby Pr...",http://ecx.images-amazon.com/images/I/41JTX248...,134.50,,Electronics,Home Audio,Stereo Components,Receivers & Amplifiers,Component Receivers,,"[pioneer, receiver, discontinued, manufacturer...",pioneer receiver discontinued manufacturer do...
3,Pioneer VSX-108 Audio/Video Receiver (Disconti...,With 50 watts per channel and surround conveni...,http://ecx.images-amazon.com/images/I/41AJK34P...,134.50,,Electronics,Home Audio,Stereo Components,Receivers & Amplifiers,Component Receivers,,"[pioneer, receiver, discontinued, manufacturer...",pioneer receiver discontinued manufacturer wa...
4,Panasonic SC-AK18 CD Compact Stereo System (Di...,"Equipped with a five-CD changer, Panasonic's S...",http://ecx.images-amazon.com/images/I/41E7DVXM...,239.95,,Electronics,Home Audio,Compact Stereos,,,,"[panasonic, cd, compact, stereo, system, disco...",panasonic cd compact stereo system discontinu...


In [ ]:
len(train)

5067

In [ ]:
#feature extraction
tfidf_vect=TfidfVectorizer(max_features=5000)
tfidf_vect.fit(train['final_lemmatized_string'])

X_train_tfidf = tfidf_vect.transform(train['final_lemmatized_string'])
X_new_tfidf = tfidf_vect.transform(test['final_lemmatized_string'])
#X_new_val = tfidf_vect.transform(val['final_lemmatized_string'])



In [ ]:

 # Combine labels from train, test, and validation sets
all_labels = np.concatenate((y_text_train, y_text_test), axis=0)

# Initialize and fit the label encoder
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)
encoded_labels_train = label_encoder.transform(y_text_train)
encoded_labels_train = to_categorical(encoded_labels_train, num_classes=len(label_encoder.classes_))

encoded_labels_test = label_encoder.transform(y_text_test)
encoded_labels_test = to_categorical(encoded_labels_test, num_classes=len(label_encoder.classes_))

In [ ]:
# Define a custom Keras regressor compatible with scikit-learn
class CustomKerasRegressor(BaseEstimator):
    def __init__(self, create_model, epochs=65, batch_size=10):
        self.create_model = create_model
        self.epochs = epochs
        self.batch_size = batch_size

    def fit(self, X, y):
        self.model = self.create_model()
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size)
        return self

    def predict(self, X):
        return self.model.predict(X)

    def score(self, X, y):
        # Return negative mean squared error (MSE) as a scoring metric
        return -self.model.evaluate(X, y)

In [ ]:

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

from sklearn.base import BaseEstimator
class CustomKerasClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, create_model, epochs=65, batch_size=10):
        self.create_model = create_model
        self.epochs = epochs
        self.batch_size = batch_size

    def fit(self, X, y):
        self.model = self.create_model()
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size)
        return self

    def predict(self, X):
        return self.model.predict(X)

    def score(self, X, y):
        return self.model.evaluate(X, y)


# Define a function to create your DAE model
def create_dae_model():
    input_dim = X_train_tfidf.shape[1]
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(512, activation='relu')(input_layer)
    encoded = Dropout(0.3)(encoded)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)
    dae = Model(input_layer, decoded)
    dae.compile(optimizer='adam', loss='binary_crossentropy')
    return dae

# Instantiate your custom Keras regressor
custom_dae_regressor = CustomKerasClassifier(create_model=create_dae_model, epochs=70, batch_size=10)

# Perform cross-validation
scores = cross_val_score(custom_dae_regressor, X_train_tfidf.toarray(), X_train_tfidf.toarray(), cv=3)
print("Cross-Validation Scores:", scores)
print("Mean Cross-Validation Score:", scores.mean())

# Fit the model on the entire dataset and save it
custom_dae_regressor.fit(X_train_tfidf.toarray(), X_train_tfidf.toarray())

Epoch 1/70
338/338 [==============================] - 4s 3ms/step - loss: 0.0813
Epoch 2/70
338/338 [==============================] - 1s 3ms/step - loss: 0.0088
Epoch 3/70
338/338 [==============================] - 1s 3ms/step - loss: 0.0080
Epoch 4/70
338/338 [==============================] - 1s 3ms/step - loss: 0.0076
Epoch 5/70
338/338 [==============================] - 1s 3ms/step - loss: 0.0075
Epoch 6/70
338/338 [==============================] - 1s 3ms/step - loss: 0.0073
Epoch 7/70
338/338 [==============================] - 1s 3ms/step - loss: 0.0072
Epoch 8/70
338/338 [==============================] - 1s 3ms/step - loss: 0.0071
Epoch 9/70
338/338 [==============================] - 1s 3ms/step - loss: 0.0070
Epoch 10/70
338/338 [==============================] - 1s 3ms/step - loss: 0.0068
Epoch 11/70
338/338 [==============================] - 1s 3ms/step - loss: 0.0066
Epoch 12/70
338/338 [==============================] - 1s 3ms/step - loss: 0.0065
Epoch 13/70
338/338 [====

CustomKerasClassifier(create_model=<function create_dae_model at 0x7d3102666d40>,
                      epochs=70)

In [ ]:
# Get the encoder part of your autoencoder model
encoder = Model(inputs=custom_dae_regressor.model.input, outputs=custom_dae_regressor.model.layers[1].output)

# Encode your train data
X_train_encoded = encoder.predict(X_train_tfidf.toarray())

# Encode your test data
X_test_encoded = encoder.predict(X_new_tfidf.toarray())

37/37 [==============================] - 0s 1ms/step


In [ ]:
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
X_Train_Images=[]
X_Test_Images=[]
for i in tqdm(range(test.shape[0])):
    img=image.load_img('./test_images/test_images/'+str(i)+'.jpg',target_size=(200,200,3))
    img=image.img_to_array(img)
    img=img/255
    X_Test_Images.append(img)
X_test_img=np.array(X_Test_Images)
lst=[]
for i in range(train.shape[0]):
    img='./train_images/train_images/'+str(i)+'.jpg'
    lst.append(img)
#print(lst)
train['image']=lst
lst=[]
for i in range(test.shape[0]):
    img='./test_images/test_images/'+str(i)+'.jpg'
    lst.append(img)
#print(lst)
test['image']=lst
target_size = (200, 200)

X_Train_Images = []  # List to store resized images

# Load and preprocess the training set images
for i in tqdm(range(train.shape[0])):
    img = image.load_img('./train_images/train_images/' + str(i) + '.jpg', target_size=(200,200,3))
    img = image.img_to_array(img)
    img = img / 255
    X_Train_Images.append(img)

# Convert the list of images to a NumPy array
X_train_img = np.array(X_Train_Images)



100%|██████████| 5067/5067 [00:04<00:00, 1026.61it/s]


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, concatenate, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2

# Define your CNN model for image input with dropout regularization, L2 regularization, and batch normalization
def create_cnn_model():
    cnn_input = Input(shape=(200, 200, 3))
    cnn = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001))(cnn_input)  # Add L2 regularization
    cnn = BatchNormalization()(cnn)  # Add BatchNormalization layer
    cnn = MaxPooling2D((2, 2))(cnn)
    cnn = Flatten()(cnn)
    cnn = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(cnn)  # Add L2 regularization
    cnn = BatchNormalization()(cnn)  # Add BatchNormalization layer
    cnn = Dropout(0.5)(cnn)  # Dropout regularization
    cnn = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(cnn)  # Add L2 regularization
    return Model(inputs=cnn_input, outputs=cnn)

# Define your DBN model for text input with dropout regularization, L2 regularization, and batch normalization
def create_dbn_model(input_dim):
    dbn_input = Input(shape=(input_dim,))
    dbn = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(dbn_input)  # Add L2 regularization
    dbn = BatchNormalization()(dbn)  # Add BatchNormalization layer
    dbn = Dropout(0.5)(dbn)  # Dropout regularization
    dbn = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(dbn)  # Add L2 regularization
    return Model(inputs=dbn_input, outputs=dbn)

# Combine both models
def create_multi_modal_model(cnn_model, dbn_model, num_classes):
    combined_input = concatenate([cnn_model.output, dbn_model.output])
    combined = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(combined_input)  # Add L2 regularization
    combined = BatchNormalization()(combined)  # Add BatchNormalization layer
    combined = Dropout(0.5)(combined)  # Dropout regularization
    combined = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(combined)  # Add L2 regularization
    output = Dense(num_classes, activation='softmax')(combined)
    return Model(inputs=[cnn_model.input, dbn_model.input], outputs=output)


In [ ]:
from tensorflow.keras.metrics import Accuracy, Precision, Recall, F1Score
from tensorflow.keras.optimizers import Adam

def train_combined_model(num_classes, X_train_img, X_train_encoded, encoded_labels_train, wp, size, epochs):
    # Create CNN model
    cnn_model = create_cnn_model()
    cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Create DBN model
    input_dim_dbn = X_train_encoded.shape[1]
    dbn_model = create_dbn_model(input_dim_dbn)
    dbn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Create multi-modal model
    multi_modal_model = create_multi_modal_model(cnn_model, dbn_model, num_classes)
    multi_modal_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Compile the model
    #multi_modal_model.compile(optimizer=Adam(), loss='categorical_crossentropy',
     #                     metrics=[Accuracy(), Precision(), Recall(), F1Score()])
    # Train the multi-modal model using both image and text inputs
    multi_modal_model.fit([X_train_img, X_train_encoded], encoded_labels_train, epochs=epochs, batch_size=size)

    # Return the trained multi-modal model
    return multi_modal_model

In [ ]:
# Train the combined model
# Define the number of classes, input data, and hyperparameters
num_classes = 11  # Example: Number of classes
 # Example: Training encoded data

wp =0.01  # Example: Some value for wp
size = 15  # Example: Some value for size
epochs = 30  # Number of epochs for training
trained_multi_modal_model = train_combined_model(num_classes, X_train_img, X_train_encoded, encoded_labels_train, wp, size, epochs=75)
# Use the trained model to make predictions on the test data


Epoch 1/75
338/338 [==============================] - 8s 14ms/step - loss: 3.1774 - accuracy: 0.6304
Epoch 2/75
338/338 [==============================] - 5s 14ms/step - loss: 2.2891 - accuracy: 0.8099
Epoch 3/75
338/338 [==============================] - 5s 14ms/step - loss: 2.0947 - accuracy: 0.8417
Epoch 4/75
338/338 [==============================] - 5s 14ms/step - loss: 1.8077 - accuracy: 0.8617
Epoch 5/75
338/338 [==============================] - 5s 14ms/step - loss: 1.5672 - accuracy: 0.8755
Epoch 6/75
338/338 [==============================] - 5s 14ms/step - loss: 1.4760 - accuracy: 0.8820
Epoch 7/75
338/338 [==============================] - 5s 14ms/step - loss: 1.4612 - accuracy: 0.8812
Epoch 8/75
338/338 [==============================] - 5s 14ms/step - loss: 1.3441 - accuracy: 0.8826
Epoch 9/75
338/338 [==============================] - 5s 14ms/step - loss: 1.2876 - accuracy: 0.8978
Epoch 10/75
338/338 [==============================] - 5s 14ms/step - loss: 1.2877 - accura

In [ ]:
import gc
gc.collect()

7392

In [ ]:
from sklearn.metrics import precision_score, recall_score,f1_score


In [ ]:
predictions = trained_multi_modal_model.predict([X_test_img, X_test_encoded])
# Evaluate the multi-modal model
loss, accuracy = trained_multi_modal_model.evaluate([X_test_img, X_test_encoded], encoded_labels_test)
print(f'Test Accuracy: {accuracy}')


# Calculate precision
# Convert predictions to multilabel-indicator format
predicted_labels_multilabel = np.eye(num_classes)[predictions.argmax(axis=1)]

# Now you can calculate precision using the multilabel-indicator format
test_precision = precision_score(encoded_labels_test, predicted_labels_multilabel, average='weighted')
print(f'Test Precision: {test_precision}')
# Calculate recall
test_recall = recall_score(encoded_labels_test,predicted_labels_multilabel, average='weighted')

print(f'Test Recall: {test_recall}')
# Calculate F1-score
test_f1_score = f1_score(encoded_labels_test,predicted_labels_multilabel, average='weighted')
print(f'Test F1-score: {test_f1_score}')
predicted_class_indices = np.argmax(predictions, axis=1)

predicted_labels = label_encoder.inverse_transform(predicted_class_indices)



37/37 [==============================] - 1s 9ms/step - loss: 0.6961 - accuracy: 0.9264
Test Accuracy: 0.9264328479766846
Test Precision: 0.9265217932203312
Test Recall: 0.9264328485885372
Test F1-score: 0.9259404445912146


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [ ]:


test['pcategories__2']=predicted_labels



In [ ]:
import gc
gc.collect()

7219

In [ ]:
file_path = 'predictions_precision.csv'

# Save the modified 'test' DataFrame to a CSV file without the DataFrame index
test.to_csv(file_path, index=False)

In [ ]:

test=pd.read_csv(r'.\predictions.csv')
test=test.replace(np.nan, '',regex=True)

FileNotFoundError: [Errno 2] No such file or directory: '.\\Dataset\\newtraindata1.csv'

In [ ]:

leafnode=leafnode_extraction(train)
leafnode=[x.lower() for x in leafnode]
testleafnode=leafnode_extraction(test)
testleafnode=[x.lower() for x in testleafnode]

train['leaf']=leafnode
test['leaf']=testleafnode

test.head()

# to train leafnode prediction
# extraction of train samples of particular level1 category

inp1=train.loc[train['categories__2']=='Computers & Accessories']
inp2=train.loc[train['categories__2']=='Accessories & Supplies']
inp3=train.loc[train['categories__2']=='GPS & Navigation']
inp4=train.loc[train['categories__2']=='eBook Readers & Accessories']
inp5=train.loc[train['categories__2']=='Car & Vehicle Electronics']
inp6=train.loc[train['categories__2']=='Camera & Photo']
inp7=train.loc[train['categories__2']=='Portable Audio & Video']
inp8=train.loc[train['categories__2']=='Cell Phones & Accessories']
inp9=train.loc[train['categories__2']=='Television & Video']
inp10=train.loc[train['categories__2']=='Home Audio']
inp11=train.loc[train['categories__2']=='Security & Surveillance']




#test
testinp1=test.loc[test['pcategories__2']=='computers & accessories']
testinp2=test.loc[test['pcategories__2']=='accessories & supplies']
testinp3=test.loc[test['pcategories__2']=='gps & navigation']
testinp4=test.loc[test['pcategories__2']=='ebook readers & accessories']
testinp5=test.loc[test['pcategories__2']=='car & vehicle electronics']
testinp6=test.loc[test['pcategories__2']=='camera & photo']
testinp7=test.loc[test['pcategories__2']=='portable audio & video']
testinp8=test.loc[test['pcategories__2']=='cell phones & accessories']
testinp9=test.loc[test['pcategories__2']=='television & video']
testinp10=test.loc[test['pcategories__2']=='home audio']
testinp11=test.loc[test['pcategories__2']=='security & surveillance']
#//////////////////////////////////////////

In [ ]:
len(testinp1)+len(testinp2)+len(testinp3)+len(testinp4)+len(testinp5)+len(testinp6)+len(testinp7)+len(testinp8)+len(testinp9)+len(testinp10)+len(testinp11)

1169

In [ ]:
def encode_labels(train_labels, test_labels):
    # Combine labels from train, test, and validation sets
    all_labels = np.concatenate((train_labels, test_labels), axis=0)

    # Initialize and fit the label encoder
    label_encoder = LabelEncoder()
    label_encoder.fit(all_labels)
    encoded_labels_train = label_encoder.transform(train_labels)
    encoded_labels_train = to_categorical(encoded_labels_train, num_classes=len(label_encoder.classes_))

    encoded_labels_test = label_encoder.transform(test_labels)
    encoded_labels_test = to_categorical(encoded_labels_test, num_classes=len(label_encoder.classes_))

    return encoded_labels_train, encoded_labels_test, label_encoder

def Load_images(inp):
    X_Images=[]
    for i in list(inp['image']):
        img=image.load_img(i,target_size=(200,200,3))
        img=image.img_to_array(img)
        img=img/255
        X_Images.append(img)
    x_images=np.array(X_Images)
    return x_images


In [ ]:
test.head()

,title,description,imUrl,price,brand,categories__1,categories__2,categories__3,categories__4,categories__5,categories__6,lemmatized_text,final_lemmatized_string,image,pcategories__2,leaf
0,Panasonic SC-AK29 Compact Stereo System (Disco...,"Equipped with a five-CD changer, Panasonic's S...",http://ecx.images-amazon.com/images/I/41QGRQSQ...,134.50,,Electronics,Home Audio,Compact Stereos,,,,"[panasonic, compact, stereo, system, discontin...",panasonic compact stereo system discontinued ...,./test_images/0.jpg,home audio,compact stereos
1,Sennheiser RS4-9 On-Ear Wireless Headphone,Sennheiser's RS4-9 wireless headphones consist...,http://g-ecx.images-amazon.com/images/G/01/x-s...,134.50,,Electronics,Accessories & Supplies,Audio & Video Accessories,Headphones,,,"[sennheiser, wireless, headphone, sennheiser, ...",sennheiser wireless headphone sennheiser wire...,./test_images/1.jpg,accessories & supplies,headphones
2,Pioneer VSX-D309 Audio/Video Receiver (Discont...,"With built-in Dolby Digital, DTS, and Dolby Pr...",http://ecx.images-amazon.com/images/I/41JTX248...,134.50,,Electronics,Home Audio,Stereo Components,Receivers & Amplifiers,Component Receivers,,"[pioneer, receiver, discontinued, manufacturer...",pioneer receiver discontinued manufacturer do...,./test_images/2.jpg,home audio,component receivers
3,Pioneer VSX-108 Audio/Video Receiver (Disconti...,With 50 watts per channel and surround conveni...,http://ecx.images-amazon.com/images/I/41AJK34P...,134.50,,Electronics,Home Audio,Stereo Components,Receivers & Amplifiers,Component Receivers,,"[pioneer, receiver, discontinued, manufacturer...",pioneer receiver discontinued manufacturer wa...,./test_images/3.jpg,home audio,component receivers
4,Panasonic SC-AK18 CD Compact Stereo System (Di...,"Equipped with a five-CD changer, Panasonic's S...",http://ecx.images-amazon.com/images/I/41E7DVXM...,239.95,,Electronics,Home Audio,Compact Stereos,,,,"[panasonic, cd, compact, stereo, system, disco...",panasonic cd compact stereo system discontinu...,./test_images/4.jpg,home audio,compact stereos


In [ ]:
#--------------------------------------------------------------------
# Instantiate DBN model for text
#Applying Model for each category__2 categories
# Model for category  'Computers & Accessories'


import gc
gc.collect()
print(' Result of Computers & Accessories:')
encoded_labels_train, encoded_labels_test,label_encoder=encode_labels(inp1['leaf'],testinp1['leaf'])
X_train_leaf1,X_test_leaf1=tf_idf_vectorization(inp1,testinp1)
# Encode your train data
X_train_encoded = encoder.predict(X_train_leaf1.toarray())
# Encode your test data
X_test_encoded = encoder.predict(X_test_leaf1.toarray())
input_dim_dbn = X_train_encoded.shape[1]
#test images loading
X_test_img=Load_images(testinp1)
X_train_img=Load_images(inp1)
# Instantiate CNN model for image
# Instantiate DBN model for text
input_dim_dbn = X_train_encoded.shape[1]
num_classes = 123 # Set the input dimensions based on the DBN input shape
epochs=50
size=5


model = train_combined_model(num_classes, X_train_img, X_train_encoded, encoded_labels_train, wp, size, epochs)


predictions = model.predict([X_test_img, X_test_encoded])
# Evaluate the multi-modal model
loss, accuracy = model.evaluate([X_test_img, X_test_encoded], encoded_labels_test)
print(f'Test Accuracy: {accuracy}')


# Calculate precision
# Convert predictions to multilabel-indicator format
predicted_labels_multilabel = np.eye(num_classes)[predictions.argmax(axis=1)]

# Now you can calculate precision using the multilabel-indicator format
test_precision = precision_score(encoded_labels_test, predicted_labels_multilabel, average='weighted')
print(f'Test Precision: {test_precision}')
# Calculate recall
test_recall = recall_score(encoded_labels_test,predicted_labels_multilabel, average='weighted')

print(f'Test Recall: {test_recall}')
# Calculate F1-score
test_f1_score = f1_score(encoded_labels_test,predicted_labels_multilabel, average='weighted')
print(f'Test F1-score: {test_f1_score}')
predicted_class_indices = np.argmax(predictions, axis=1)

predicted_labels = label_encoder.inverse_transform(predicted_class_indices)







 Result of Computers & Accessories:
(2029, 5000)
(431, 5000)
14/14 [==============================] - 0s 2ms/step
Epoch 1/50
406/406 [==============================] - 10s 18ms/step - loss: 3.9118 - accuracy: 0.1986
Epoch 2/50
406/406 [==============================] - 7s 17ms/step - loss: 3.1337 - accuracy: 0.2755
Epoch 3/50
406/406 [==============================] - 7s 18ms/step - loss: 2.9233 - accuracy: 0.3238
Epoch 4/50
406/406 [==============================] - 7s 18ms/step - loss: 2.7174 - accuracy: 0.3420
Epoch 5/50
406/406 [==============================] - 7s 18ms/step - loss: 2.5408 - accuracy: 0.3775
Epoch 6/50
406/406 [==============================] - 7s 18ms/step - loss: 2.4001 - accuracy: 0.4110
Epoch 7/50
406/406 [==============================] - 7s 18ms/step - loss: 2.2506 - accuracy: 0.4209
Epoch 8/50
406/406 [==============================] - 7s 17ms/step - loss: 2.1293 - accuracy: 0.4515
Epoch 9/50
406/406 [==============================] - 7s 17ms/step - loss: 2.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [ ]:
# Display predicted labels
testinp1['pleaf']=predicted_labels
file_path = 'testinp111.csv'

# Save the modified 'test' DataFrame to a CSV file without the DataFrame index
testinp1.to_csv(file_path, index=False)

C:\Users\sowjanya\AppData\Local\Temp\ipykernel_24796\338734342.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testinp1['pleaf']=predicted_labels


In [ ]:
# Model for category  'Accessories & Supplies'

print(' Result of Accessories & Supplies:')

encoded_labels_train, encoded_labels_test,label_encoder=encode_labels(inp2['leaf'],testinp2['leaf'])
X_train_leaf,X_test_leaf=tf_idf_vectorization(inp2,testinp2)
# Encode your train data
X_train_encoded = encoder.predict(X_train_leaf.toarray())
# Encode your test data
X_test_encoded = encoder.predict(X_test_leaf.toarray())
input_dim_dbn = X_train_encoded.shape[1]
#test images loading
X_test_img=Load_images(testinp2)
X_train_img=Load_images(inp2)

input_dim_dbn = X_train_encoded.shape[1]
num_classes = 142 # Set the input dimensions based on the DBN input shape
ep=50
size=5
model = train_combined_model(num_classes, X_train_img, X_train_encoded, encoded_labels_train, wp, size, epochs)


predictions = model.predict([X_test_img, X_test_encoded])
# Evaluate the multi-modal model
loss, accuracy = model.evaluate([X_test_img, X_test_encoded], encoded_labels_test)
print(f'Test Accuracy: {accuracy}')


# Calculate precision
# Convert predictions to multilabel-indicator format
predicted_labels_multilabel = np.eye(num_classes)[predictions.argmax(axis=1)]

# Now you can calculate precision using the multilabel-indicator format
test_precision = precision_score(encoded_labels_test, predicted_labels_multilabel, average='weighted')
print(f'Test Precision: {test_precision}')
# Calculate recall
test_recall = recall_score(encoded_labels_test,predicted_labels_multilabel, average='weighted')

print(f'Test Recall: {test_recall}')
# Calculate F1-score
test_f1_score = f1_score(encoded_labels_test,predicted_labels_multilabel, average='weighted')
print(f'Test F1-score: {test_f1_score}')
predicted_class_indices = np.argmax(predictions, axis=1)

predicted_labels = label_encoder.inverse_transform(predicted_class_indices


# Display predicted labels
#testinp2['pleaf']=predicted_labels

 Result of Accessories & Supplies:
(707, 4228)
(533, 4228)


Epoch 1/50

142/142 [==============================] - 5s 26ms/step - loss: 0.2052 - val_loss: 0.0149
Epoch 2/50
142/142 [==============================] - 3s 22ms/step - loss: 0.0107 - val_loss: 0.0106
Epoch 3/50
142/142 [==============================] - 3s 19ms/step - loss: 0.0091 - val_loss: 0.0098
Epoch 4/50
142/142 [==============================] - 3s 19ms/step - loss: 0.0086 - val_loss: 0.0094
Epoch 5/50
142/142 [==============================] - 3s 18ms/step - loss: 0.0083 - val_loss: 0.0092
Epoch 6/50
142/142 [==============================] - 3s 19ms/step - loss: 0.0081 - val_loss: 0.0090
Epoch 7/50
142/142 [==============================] - 2s 18ms/step - loss: 0.0079 - val_loss: 0.0089
Epoch 8/50
142/142 [==============================] - 3s 18ms/step - loss: 0.0078 - val_loss: 0.0088
Epoch 9/50
142/142 [==============================] - 3s 18ms/step - loss: 0.0078 - val_loss: 0.0088
Epoch 10/50
142/142 [========

In [ ]:
# Display predicted labels
#43
testinp2['pleaf']=predicted_labels
file_path = 'testinp22.csv'

# Save the modified 'test' DataFrame to a CSV file without the DataFrame index
testinp2.to_csv(file_path, index=False)

C:\Users\sowjanya\AppData\Local\Temp\ipykernel_44876\1053271661.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testinp2['pleaf']=predicted_labels


In [ ]:
# Model for category  'GPS & Navigation'
import gc
gc.collect()

6533

In [ ]:
keras.backend.clear_session()

In [ ]:

print(' Result of GPS & Navigation:')

encoded_labels_train, encoded_labels_test, encoded_labels_val,label_encoder=encode_labels(inp3['leaf'],testinp3['leaf'],val3['leaf'])
X_train_leaf,X_test_leaf,X_val_leaf=tf_idf_vectorization(inp3,testinp3,val3)
X_train_encoded,X_test_encoded,X_val_encoded=train_denoising_autoencoder(X_train_leaf, X_test_leaf, X_val_leaf)
input_dim_dbn = X_train_encoded.shape[1]
#test images loading
X_test_img=Load_images(testinp3)
X_train_img=Load_images(inp3)
X_val_img=Load_images(val3)
input_dim_dbn = X_train_encoded.shape[1]
num_classes = 13 # Set the input dimensions based on the DBN input shape
ep=55
size=3
model= cnn_dbn(num_classes,X_train_img,X_train_encoded,encoded_labels_train,ep,size,X_val_img,X_val_encoded,encoded_labels_val)

# Evaluate the multi-modal model
loss, accuracy = model.evaluate([X_test_img, X_test_encoded], encoded_labels_test)
print(f'Test Accuracy: {accuracy}')
predicted_probabilities= model.predict([X_test_img, X_test_encoded])
# Convert probabilities to class indices (argmax to get the index of the highest probability)
predicted_class_indices = np.argmax(predicted_probabilities, axis=1)

# Convert class indices back to original labels using inverse_transform of label_encoder
predicted_labels = label_encoder.inverse_transform(predicted_class_indices)





 Result of GPS & Navigation:
(45, 943)
(11, 943)
Epoch 1/50
9/9 [==============================] - 1s 28ms/step - loss: 0.6803 - val_loss: 0.6617
Epoch 2/50
9/9 [==============================] - 0s 18ms/step - loss: 0.6255 - val_loss: 0.5992
Epoch 3/50
9/9 [==============================] - 0s 16ms/step - loss: 0.5108 - val_loss: 0.4889
Epoch 4/50
9/9 [==============================] - 0s 17ms/step - loss: 0.3457 - val_loss: 0.3390
Epoch 5/50
9/9 [==============================] - 0s 18ms/step - loss: 0.1891 - val_loss: 0.2065
Epoch 6/50
9/9 [==============================] - 0s 16ms/step - loss: 0.0989 - val_loss: 0.1242
Epoch 7/50
9/9 [==============================] - 0s 16ms/step - loss: 0.0607 - val_loss: 0.0852
Epoch 8/50
9/9 [==============================] - 0s 16ms/step - loss: 0.0483 - val_loss: 0.0663
Epoch 9/50
9/9 [==============================] - 0s 16ms/step - loss: 0.0424 - val_loss: 0.0572
Epoch 10/50
9/9 [==============================] - 0s 14ms/step - loss: 0.0395

In [ ]:
#2727
# Display predicted labels
testinp3['pleaf']=predicted_labels
file_path = 'testinp33.csv'

# Save the modified 'test' DataFrame to a CSV file without the DataFrame index
testinp3.to_csv(file_path, index=False)

C:\Users\sowjanya\AppData\Local\Temp\ipykernel_44876\1714627473.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testinp3['pleaf']=predicted_labels


In [ ]:
encoded_labels_train, encoded_labels_test, encoded_labels_val,label_encoder1=encode_labels(inp4['leaf'],testinp4['leaf'],val4['leaf'])
len(label_encoder1.classes_)

180

In [ ]:
# Model for category  'GPS & Navigation'
import gc
gc.collect()

9573

In [ ]:
# Model for category  'eBook Readers & Accessories'

print(' Result of eBook Readers & Accessories:')

encoded_labels_train, encoded_labels_test, encoded_labels_val,label_encoder=encode_labels(inp4['leaf'],testinp4['leaf'],val4['leaf'])
X_train_leaf,X_test_leaf,X_val_leaf=tf_idf_vectorization(inp4,testinp4,val4)
X_train_encoded,X_test_encoded,X_val_encoded=train_denoising_autoencoder(X_train_leaf, X_test_leaf, X_val_leaf)
input_dim_dbn = X_train_encoded.shape[1]
#test images loading
X_test_img=Load_images(testinp4)
X_train_img=Load_images(inp4)
X_val_img=Load_images(val4)
input_dim_dbn = X_train_encoded.shape[1]
num_classes = 13 # Set the input dimensions based on the DBN input shape
ep=50
size=5
model= cnn_dbn(num_classes,X_train_img,X_train_encoded,encoded_labels_train,ep,size,X_val_img,X_val_encoded,encoded_labels_val)

# Evaluate the multi-modal model
loss, accuracy = model.evaluate([X_test_img, X_test_encoded], encoded_labels_test)
print(f'Test Accuracy: {accuracy}')
predicted_probabilities= model.predict([X_test_img, X_test_encoded])
# Convert probabilities to class indices (argmax to get the index of the highest probability)
predicted_class_indices = np.argmax(predicted_probabilities, axis=1)

# Convert class indices back to original labels using inverse_transform of label_encoder
predicted_labels = label_encoder.inverse_transform(predicted_class_indices)




testinp4['pleaf']=predicted_labels

 Result of eBook Readers & Accessories:


ValueError: Found array with 0 sample(s) (shape=(0, 864)) while a minimum of 1 is required by TfidfTransformer.

In [ ]:
print(testinp4.shape)

(0, 15)


45

In [ ]:
# Model for category  'Car & Vehicle Electronics'

print(' Result of car & vehicle electronics:')

if len(testinp5)!=0:
    encoded_labels_train, encoded_labels_test, encoded_labels_val,label_encoder=encode_labels(inp5['leaf'],testinp5['leaf'],val5['leaf'])
    X_train_leaf,X_test_leaf,X_val_leaf=tf_idf_vectorization(inp5,testinp5,val5)
    X_train_encoded,X_test_encoded,X_val_encoded=train_denoising_autoencoder(X_train_leaf, X_test_leaf, X_val_leaf)
    input_dim_dbn = X_train_encoded.shape[1]
    #test images loading
    X_test_img=Load_images(testinp5)
    X_train_img=Load_images(inp5)
    X_val_img=Load_images(val5)
    input_dim_dbn = X_train_encoded.shape[1]
    num_classes = 45 # Set the input dimensions based on the DBN input shape
    ep=50
    size=5
    model= cnn_dbn(num_classes,X_train_img,X_train_encoded,encoded_labels_train,ep,size,X_val_img,X_val_encoded,encoded_labels_val)

    # Evaluate the multi-modal model
    loss, accuracy = model.evaluate([X_test_img, X_test_encoded], encoded_labels_test)
    print(f'Test Accuracy: {accuracy}')
    predicted_probabilities= model.predict([X_test_img, X_test_encoded])
    # Convert probabilities to class indices (argmax to get the index of the highest probability)
    predicted_class_indices = np.argmax(predicted_probabilities, axis=1)

    # Convert class indices back to original labels using inverse_transform of label_encoder
    predicted_labels = label_encoder.inverse_transform(predicted_class_indices)



 Result of car & vehicle electronics:
(116, 1582)
(21, 1582)
Epoch 1/50
24/24 [==============================] - 1s 13ms/step - loss: 0.6366 - val_loss: 0.5566
Epoch 2/50
24/24 [==============================] - 0s 12ms/step - loss: 0.3112 - val_loss: 0.2065
Epoch 3/50
24/24 [==============================] - 0s 7ms/step - loss: 0.0761 - val_loss: 0.0648
Epoch 4/50
24/24 [==============================] - 0s 8ms/step - loss: 0.0322 - val_loss: 0.0392
Epoch 5/50
24/24 [==============================] - 0s 7ms/step - loss: 0.0246 - val_loss: 0.0318
Epoch 6/50
24/24 [==============================] - 0s 10ms/step - loss: 0.0217 - val_loss: 0.0287
Epoch 7/50
24/24 [==============================] - 0s 11ms/step - loss: 0.0203 - val_loss: 0.0268
Epoch 8/50
24/24 [==============================] - 0s 11ms/step - loss: 0.0193 - val_loss: 0.0257
Epoch 9/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0188 - val_loss: 0.0249
Epoch 10/50
24/24 [==============================] -

In [ ]:
#52
# Display predicted labels

file_path = 'testinp55.csv'


# Display predicted labels
testinp5['pleaf']=predicted_labels
# Save the modified 'test' DataFrame to a CSV file without the DataFrame index
testinp5.to_csv(file_path, index=False)

C:\Users\sowjanya\AppData\Local\Temp\ipykernel_44876\1589850183.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testinp5['pleaf']=predicted_labels


In [ ]:
# Model for category  'GPS & Navigation'
import gc
gc.collect()
encoded_labels_train, encoded_labels_test, encoded_labels_val,label_encoder1=encode_labels(inp6['leaf'],testinp6['leaf'],val6['leaf'])
len(label_encoder1.classes_)

104

In [ ]:
# Model for category  'Camera & Photo'

print('Result of Camera & Photo:')

if len(testinp6)!=0:
    encoded_labels_train, encoded_labels_test, encoded_labels_val,label_encoder=encode_labels(inp6['leaf'],testinp6['leaf'],val6['leaf'])
    X_train_leaf,X_test_leaf,X_val_leaf=tf_idf_vectorization(inp6,testinp6,val6)
    X_train_encoded,X_test_encoded,X_val_encoded=train_denoising_autoencoder(X_train_leaf, X_test_leaf, X_val_leaf)
    input_dim_dbn = X_train_encoded.shape[1]
    #test images loading
    X_test_img=Load_images(testinp6)
    X_train_img=Load_images(inp6)
    X_val_img=Load_images(val6)
    input_dim_dbn = X_train_encoded.shape[1]
    num_classes = 104 # Set the input dimensions based on the DBN input shape
    ep=70
    size=5
    model= cnn_dbn(num_classes,X_train_img,X_train_encoded,encoded_labels_train,ep,size,X_val_img,X_val_encoded,encoded_labels_val)

    # Evaluate the multi-modal model
    loss, accuracy = model.evaluate([X_test_img, X_test_encoded], encoded_labels_test)
    print(f'Test Accuracy: {accuracy}')
    predicted_probabilities= model.predict([X_test_img, X_test_encoded])
    # Convert probabilities to class indices (argmax to get the index of the highest probability)
    predicted_class_indices = np.argmax(predicted_probabilities, axis=1)

    # Convert class indices back to original labels using inverse_transform of label_encoder
    predicted_labels = label_encoder.inverse_transform(predicted_class_indices)


Result of Camera & Photo:
(497, 4145)
(487, 4145)
Epoch 1/50
100/100 [==============================] - 4s 28ms/step - loss: 0.2761 - val_loss: 0.0217
Epoch 2/50
100/100 [==============================] - 2s 22ms/step - loss: 0.0141 - val_loss: 0.0125
Epoch 3/50
100/100 [==============================] - 2s 19ms/step - loss: 0.0111 - val_loss: 0.0109
Epoch 4/50
100/100 [==============================] - 2s 19ms/step - loss: 0.0103 - val_loss: 0.0105
Epoch 5/50
100/100 [==============================] - 2s 19ms/step - loss: 0.0099 - val_loss: 0.0103
Epoch 6/50
100/100 [==============================] - 2s 19ms/step - loss: 0.0095 - val_loss: 0.0101
Epoch 7/50
100/100 [==============================] - 3s 31ms/step - loss: 0.0093 - val_loss: 0.0102
Epoch 8/50
100/100 [==============================] - 2s 23ms/step - loss: 0.0091 - val_loss: 0.0102
Epoch 9/50
100/100 [==============================] - 2s 19ms/step - loss: 0.0089 - val_loss: 0.0101
Epoch 10/50
100/100 [====================

Epoch 21/70
100/100 [==============================] - 14s 141ms/step - loss: 1.1142 - accuracy: 0.6901 - val_loss: 10.0320 - val_accuracy: 0.1186
Epoch 22/70
100/100 [==============================] - 14s 140ms/step - loss: 1.1602 - accuracy: 0.6680 - val_loss: 9.1565 - val_accuracy: 0.1186
Epoch 23/70
100/100 [==============================] - 14s 140ms/step - loss: 0.9946 - accuracy: 0.6942 - val_loss: 8.7645 - val_accuracy: 0.1356
Epoch 24/70
100/100 [==============================] - 14s 141ms/step - loss: 1.0038 - accuracy: 0.7002 - val_loss: 10.1454 - val_accuracy: 0.1356
Epoch 25/70
100/100 [==============================] - 14s 141ms/step - loss: 1.0223 - accuracy: 0.7002 - val_loss: 10.3132 - val_accuracy: 0.1356
Epoch 26/70
100/100 [==============================] - 14s 141ms/step - loss: 0.9910 - accuracy: 0.7143 - val_loss: 10.2788 - val_accuracy: 0.1356
Epoch 27/70
100/100 [==============================] - 14s 140ms/step - loss: 0.9976 - accuracy: 0.7183 - val_loss: 7.62

In [ ]:
#61
file_path = 'testinp66.csv'
# Display predicted labels
testinp6['pleaf']=predicted_labels
# Save the modified 'test' DataFrame to a CSV file without the DataFrame index
testinp6.to_csv(file_path, index=False)

C:\Users\sowjanya\AppData\Local\Temp\ipykernel_44876\2929392573.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testinp6['pleaf']=predicted_labels


In [ ]:
# Model for category  'GPS & Navigation'
import gc
gc.collect()
encoded_labels_train, encoded_labels_test, encoded_labels_val,label_encoder1=encode_labels(inp7['leaf'],testinp7['leaf'],val7['leaf'])
len(label_encoder1.classes_)

73

In [ ]:
# Model for category  'Portable Audio & Video'

print(' Result of Portable Audio & Video:')

if len(testinp7)!=0:
    encoded_labels_train, encoded_labels_test, encoded_labels_val,label_encoder=encode_labels(inp7['leaf'],testinp7['leaf'],val7['leaf'])
    X_train_leaf,X_test_leaf,X_val_leaf=tf_idf_vectorization(inp7,testinp7,val7)
    X_train_encoded,X_test_encoded,X_val_encoded=train_denoising_autoencoder(X_train_leaf, X_test_leaf, X_val_leaf)
    input_dim_dbn = X_train_encoded.shape[1]
    #test images loading
    X_test_img=Load_images(testinp7)
    X_train_img=Load_images(inp7)
    X_val_img=Load_images(val7)
    input_dim_dbn = X_train_encoded.shape[1]
    num_classes = 73 # Set the input dimensions based on the DBN input shape
    ep=47
    size=5
    model= cnn_dbn(num_classes,X_train_img,X_train_encoded,encoded_labels_train,ep,size,X_val_img,X_val_encoded,encoded_labels_val)

    # Evaluate the multi-modal model
    loss, accuracy = model.evaluate([X_test_img, X_test_encoded], encoded_labels_test)
    print(f'Test Accuracy: {accuracy}')
    predicted_probabilities= model.predict([X_test_img, X_test_encoded])
    # Convert probabilities to class indices (argmax to get the index of the highest probability)
    predicted_class_indices = np.argmax(predicted_probabilities, axis=1)

    # Convert class indices back to original labels using inverse_transform of label_encoder
    predicted_labels = label_encoder.inverse_transform(predicted_class_indices)

 Result of Portable Audio & Video:
(328, 3413)
(209, 3413)
Epoch 1/50
66/66 [==============================] - 1s 16ms/step - loss: 0.3619 - val_loss: 0.0751
Epoch 2/50
66/66 [==============================] - 1s 14ms/step - loss: 0.0233 - val_loss: 0.0286
Epoch 3/50
66/66 [==============================] - 1s 14ms/step - loss: 0.0169 - val_loss: 0.0225
Epoch 4/50
66/66 [==============================] - 1s 14ms/step - loss: 0.0153 - val_loss: 0.0203
Epoch 5/50
66/66 [==============================] - 1s 14ms/step - loss: 0.0144 - val_loss: 0.0194
Epoch 6/50
66/66 [==============================] - 1s 14ms/step - loss: 0.0138 - val_loss: 0.0188
Epoch 7/50
66/66 [==============================] - 1s 13ms/step - loss: 0.0135 - val_loss: 0.0186
Epoch 8/50
66/66 [==============================] - 1s 13ms/step - loss: 0.0131 - val_loss: 0.0181
Epoch 9/50
66/66 [==============================] - 1s 12ms/step - loss: 0.0129 - val_loss: 0.0180
Epoch 10/50
66/66 [==============================]

In [ ]:
#755
file_path = 'testinp77.csv'
# Display predicted labels
testinp7['pleaf']=predicted_labels
# Save the modified 'test' DataFrame to a CSV file without the DataFrame index
testinp7.to_csv(file_path, index=False)

C:\Users\sowjanya\AppData\Local\Temp\ipykernel_44876\3184255871.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testinp7['pleaf']=predicted_labels


In [ ]:
# Model for category  'GPS & Navigation'
import gc
gc.collect()
encoded_labels_train, encoded_labels_test, encoded_labels_val,label_encoder1=encode_labels(inp8['leaf'],testinp8['leaf'],val8['leaf'])
len(label_encoder1.classes_)

5

In [ ]:
# Model for category  'Cell Phones & Accessories'

print(' Result of Cell Phones & Accessories:')

if len(testinp8)!=0:
    encoded_labels_train, encoded_labels_test, encoded_labels_val,label_encoder=encode_labels(inp8['leaf'],testinp8['leaf'],val8['leaf'])
    X_train_leaf,X_test_leaf,X_val_leaf=tf_idf_vectorization(inp8,testinp8,val8)
    X_train_encoded,X_test_encoded,X_val_encoded=train_denoising_autoencoder(X_train_leaf, X_test_leaf, X_val_leaf)
    input_dim_dbn = X_train_encoded.shape[1]
    #test images loading
    X_test_img=Load_images(testinp8)
    X_train_img=Load_images(inp8)
    X_val_img=Load_images(val8)
    input_dim_dbn = X_train_encoded.shape[1]
    num_classes = 5 # Set the input dimensions based on the DBN input shape
    ep=50
    size=5
    model= cnn_dbn(num_classes,X_train_img,X_train_encoded,encoded_labels_train,ep,size,X_val_img,X_val_encoded,encoded_labels_val)

    # Evaluate the multi-modal model
    loss, accuracy = model.evaluate([X_test_img, X_test_encoded], encoded_labels_test)
    print(f'Test Accuracy: {accuracy}')
    predicted_probabilities= model.predict([X_test_img, X_test_encoded])
    # Convert probabilities to class indices (argmax to get the index of the highest probability)
    predicted_class_indices = np.argmax(predicted_probabilities, axis=1)

    # Convert class indices back to original labels using inverse_transform of label_encoder
    predicted_labels = label_encoder.inverse_transform(predicted_class_indices)

 Result of Cell Phones & Accessories:


In [ ]:

file_path = 'testinp8.csv'
# Display predicted labels
testinp8['pleaf']=predicted_labels
# Save the modified 'test' DataFrame to a CSV file without the DataFrame index
testinp8.to_csv(file_path, index=False)

C:\Users\sowjanya\AppData\Local\Temp\ipykernel_24796\1275924586.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testinp8['pleaf']=predicted_labels


In [ ]:
# Model for category  'GPS & Navigation'
import gc
gc.collect()
encoded_labels_train, encoded_labels_test, encoded_labels_val,label_encoder1=encode_labels(inp9['leaf'],testinp9['leaf'],val9['leaf'])
len(label_encoder1.classes_)

30

In [ ]:
# Model for category  'Television & Video'
print(' Result of Television & Video:')
if len(testinp9)!=0:
    encoded_labels_train, encoded_labels_test, encoded_labels_val,label_encoder=encode_labels(inp9['leaf'],testinp9['leaf'],val9['leaf'])
    X_train_leaf,X_test_leaf,X_val_leaf=tf_idf_vectorization(inp9,testinp9,val9)
    X_train_encoded,X_test_encoded,X_val_encoded=train_denoising_autoencoder(X_train_leaf, X_test_leaf, X_val_leaf)
    input_dim_dbn = X_train_encoded.shape[1]
    #test images loading
    X_test_img=Load_images(testinp9)
    X_train_img=Load_images(inp9)
    X_val_img=Load_images(val9)
    input_dim_dbn = X_train_encoded.shape[1]
    num_classes = 30 # Set the input dimensions based on the DBN input shape
    ep=50
    size=5
    model= cnn_dbn(num_classes,X_train_img,X_train_encoded,encoded_labels_train,ep,size,X_val_img,X_val_encoded,encoded_labels_val)

    # Evaluate the multi-modal model
    loss, accuracy = model.evaluate([X_test_img, X_test_encoded], encoded_labels_test)
    print(f'Test Accuracy: {accuracy}')
    predicted_probabilities= model.predict([X_test_img, X_test_encoded])
    # Convert probabilities to class indices (argmax to get the index of the highest probability)
    predicted_class_indices = np.argmax(predicted_probabilities, axis=1)

    # Convert class indices back to original labels using inverse_transform of label_encoder
    predicted_labels = label_encoder.inverse_transform(predicted_class_indices)

 Result of Television & Video:
(122, 2795)
(61, 2795)
Epoch 1/50
25/25 [==============================] - 1s 21ms/step - loss: 0.6274 - val_loss: 0.5435
Epoch 2/50
25/25 [==============================] - 0s 15ms/step - loss: 0.2058 - val_loss: 0.1847
Epoch 3/50
25/25 [==============================] - 0s 11ms/step - loss: 0.0410 - val_loss: 0.0893
Epoch 4/50
25/25 [==============================] - 0s 11ms/step - loss: 0.0278 - val_loss: 0.0700
Epoch 5/50
25/25 [==============================] - 0s 11ms/step - loss: 0.0250 - val_loss: 0.0620
Epoch 6/50
25/25 [==============================] - 0s 14ms/step - loss: 0.0233 - val_loss: 0.0586
Epoch 7/50
25/25 [==============================] - 0s 11ms/step - loss: 0.0223 - val_loss: 0.0540
Epoch 8/50
25/25 [==============================] - 0s 11ms/step - loss: 0.0218 - val_loss: 0.0510
Epoch 9/50
25/25 [==============================] - 0s 11ms/step - loss: 0.0208 - val_loss: 0.0499
Epoch 10/50
25/25 [==============================] - 0s

In [ ]:
#72
file_path = 'testinp99.csv'
# Display predicted labels
testinp9['pleaf']=predicted_labels
# Save the modified 'test' DataFrame to a CSV file without the DataFrame index
testinp9.to_csv(file_path, index=False)

C:\Users\sowjanya\AppData\Local\Temp\ipykernel_44876\3339020620.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testinp9['pleaf']=predicted_labels


In [ ]:
# Model for category  'GPS & Navigation'
import gc
gc.collect()
encoded_labels_train, encoded_labels_test, encoded_labels_val,label_encoder1=encode_labels(inp10['leaf'],testinp10['leaf'],val10['leaf'])
len(label_encoder1.classes_)

48

In [ ]:
# Model for category  'Home Audio'

print(' Result of Home Audio:')
if len(testinp10)!=0:
    encoded_labels_train, encoded_labels_test, encoded_labels_val,label_encoder=encode_labels(inp10['leaf'],testinp10['leaf'],val10['leaf'])
    X_train_leaf,X_test_leaf,X_val_leaf=tf_idf_vectorization(inp10,testinp10,val10)
    X_train_encoded,X_test_encoded,X_val_encoded=train_denoising_autoencoder(X_train_leaf, X_test_leaf, X_val_leaf)
    input_dim_dbn = X_train_encoded.shape[1]
    #test images loading
    X_test_img=Load_images(testinp10)
    X_train_img=Load_images(inp10)
    X_val_img=Load_images(val10)
    input_dim_dbn = X_train_encoded.shape[1]
    num_classes = 48 # Set the input dimensions based on the DBN input shape
    ep=50
    size=5
    model= cnn_dbn(num_classes,X_train_img,X_train_encoded,encoded_labels_train,ep,size,X_val_img,X_val_encoded,encoded_labels_val)

    # Evaluate the multi-modal model
    loss, accuracy = model.evaluate([X_test_img, X_test_encoded], encoded_labels_test)
    print(f'Test Accuracy: {accuracy}')
    predicted_probabilities= model.predict([X_test_img, X_test_encoded])
    # Convert probabilities to class indices (argmax to get the index of the highest probability)
    predicted_class_indices = np.argmax(predicted_probabilities, axis=1)

    # Convert class indices back to original labels using inverse_transform of label_encoder
    predicted_labels = label_encoder.inverse_transform(predicted_class_indices)

 Result of Home Audio:
(230, 3134)
(105, 3134)
Epoch 1/50
46/46 [==============================] - 1s 15ms/step - loss: 0.4929 - val_loss: 0.1895
Epoch 2/50
46/46 [==============================] - 1s 14ms/step - loss: 0.0482 - val_loss: 0.0370
Epoch 3/50
46/46 [==============================] - 0s 11ms/step - loss: 0.0202 - val_loss: 0.0267
Epoch 4/50
46/46 [==============================] - 0s 11ms/step - loss: 0.0172 - val_loss: 0.0234
Epoch 5/50
46/46 [==============================] - 0s 10ms/step - loss: 0.0158 - val_loss: 0.0217
Epoch 6/50
46/46 [==============================] - 1s 11ms/step - loss: 0.0150 - val_loss: 0.0209
Epoch 7/50
46/46 [==============================] - 1s 13ms/step - loss: 0.0144 - val_loss: 0.0204
Epoch 8/50
46/46 [==============================] - 0s 11ms/step - loss: 0.0141 - val_loss: 0.0202
Epoch 9/50
46/46 [==============================] - 1s 11ms/step - loss: 0.0137 - val_loss: 0.0197
Epoch 10/50
46/46 [==============================] - 0s 11ms/s

Epoch 22/50
46/46 [==============================] - 6s 131ms/step - loss: 0.5867 - accuracy: 0.8130 - val_loss: 10.2468 - val_accuracy: 0.0690
Epoch 23/50
46/46 [==============================] - 6s 130ms/step - loss: 0.6872 - accuracy: 0.7739 - val_loss: 7.3650 - val_accuracy: 0.0690
Epoch 24/50
46/46 [==============================] - 6s 129ms/step - loss: 0.5840 - accuracy: 0.8217 - val_loss: 10.0840 - val_accuracy: 0.0690
Epoch 25/50
46/46 [==============================] - 6s 131ms/step - loss: 0.6370 - accuracy: 0.8261 - val_loss: 9.0278 - val_accuracy: 0.0345
Epoch 26/50
46/46 [==============================] - 6s 131ms/step - loss: 0.4729 - accuracy: 0.8565 - val_loss: 10.2280 - val_accuracy: 0.0345
Epoch 27/50
46/46 [==============================] - 6s 130ms/step - loss: 0.5920 - accuracy: 0.8174 - val_loss: 11.5538 - val_accuracy: 0.0345
Epoch 28/50
46/46 [==============================] - 6s 129ms/step - loss: 0.4523 - accuracy: 0.8565 - val_loss: 10.2033 - val_accuracy: 0

In [ ]:
#56
file_path = 'testinp100.csv'
# Display predicted labels
testinp10['pleaf']=predicted_labels
# Save the modified 'test' DataFrame to a CSV file without the DataFrame index
testinp10.to_csv(file_path, index=False)

C:\Users\sowjanya\AppData\Local\Temp\ipykernel_44876\2141956935.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testinp10['pleaf']=predicted_labels


In [ ]:
# Model for category  'GPS & Navigation'
import gc
gc.collect()
encoded_labels_train, encoded_labels_test, encoded_labels_val,label_encoder1=encode_labels(inp11['leaf'],testinp11['leaf'],val11['leaf'])
len(label_encoder1.classes_)

In [ ]:
# Model for category  'Security & Surveillance'

print(' Result of Security & Surveillance:')
if len(testinp11)!=0:
    encoded_labels_train, encoded_labels_test, encoded_labels_val,label_encoder=encode_labels(inp11['leaf'],testinp11['leaf'],val11['leaf'])
    X_train_leaf,X_test_leaf,X_val_leaf=tf_idf_vectorization(inp11,testinp11,val11)
    X_train_encoded,X_test_encoded,X_val_encoded=train_denoising_autoencoder(X_train_leaf, X_test_leaf, X_val_leaf)
    input_dim_dbn = X_train_encoded.shape[1]
    #test images loading
    X_test_img=Load_images(testinp11)
    X_train_img=Load_images(inp11)
    X_val_img=Load_images(val11)
    input_dim_dbn = X_train_encoded.shape[1]
    num_classes = 11 # Set the input dimensions based on the DBN input shape
    ep=35
    size=5
    model= cnn_dbn(num_classes,X_train_img,X_train_encoded,encoded_labels_train,ep,size,X_val_img,X_val_encoded,encoded_labels_val)

    # Evaluate the multi-modal model
    loss, accuracy = model.evaluate([X_test_img, X_test_encoded], encoded_labels_test)
    print(f'Test Accuracy: {accuracy}')
    predicted_probabilities= model.predict([X_test_img, X_test_encoded])
    # Convert probabilities to class indices (argmax to get the index of the highest probability)
    predicted_class_indices = np.argmax(predicted_probabilities, axis=1)

    # Convert class indices back to original labels using inverse_transform of label_encoder
    predicted_labels = label_encoder.inverse_transform(predicted_class_indices)

 Result of Security & Surveillance:
(16, 465)
(2, 465)
Epoch 1/40
4/4 [==============================] - 0s 35ms/step - loss: 0.6886 - val_loss: 0.6768
Epoch 2/40
4/4 [==============================] - 0s 10ms/step - loss: 0.6716 - val_loss: 0.6596
Epoch 3/40
4/4 [==============================] - 0s 9ms/step - loss: 0.6488 - val_loss: 0.6392
Epoch 4/40
4/4 [==============================] - 0s 9ms/step - loss: 0.6207 - val_loss: 0.6141
Epoch 5/40
4/4 [==============================] - 0s 12ms/step - loss: 0.5850 - val_loss: 0.5839
Epoch 6/40
4/4 [==============================] - 0s 10ms/step - loss: 0.5446 - val_loss: 0.5478
Epoch 7/40
4/4 [==============================] - 0s 10ms/step - loss: 0.4836 - val_loss: 0.5057
Epoch 8/40
4/4 [==============================] - 0s 10ms/step - loss: 0.4252 - val_loss: 0.4584
Epoch 9/40
4/4 [==============================] - 0s 10ms/step - loss: 0.3591 - val_loss: 0.4067
Epoch 10/40
4/4 [==============================] - 0s 10ms/step - loss: 0.

In [ ]:
#55
file_path = 'testinp11.csv'
# Display predicted labels
testinp11['pleaf']=predicted_labels
# Save the modified 'test' DataFrame to a CSV file without the DataFrame index
testinp11.to_csv(file_path, index=False)

C:\Users\sowjanya\AppData\Local\Temp\ipykernel_44876\1120535286.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testinp11['pleaf']=predicted_labels


In [ ]:
testinp11.head()

,title,description,imUrl,price,brand,categories__1,categories__2,categories__3,categories__4,categories__5,categories__6,lemmatized_text,final_lemmatized_string,pcategories__2,image,leaf
51,Coleman Cable 05932 6-foot 18/2 SPT Reflector...,Heavy duty reflector light - UL listed. 18/2 ...,http://ecx.images-amazon.com/images/I/41rCI-dp...,16.3,,Electronics,Computers & Accessories,Cables & Accessories,Video Projector Accessories,Lamps,,"[coleman, cable, spt, reflector, light, reflec...",coleman cable spt reflector light reflector h...,security & surveillance,C:/Users/sowjanya/Documents/RMTC_IMPLEMENTATIO...,lamps
745,Uniden LRD767 All Band VG-2 Undetectability an...,,http://ecx.images-amazon.com/images/I/41W47S46...,134.5,,Electronics,Car & Vehicle Electronics,Vehicle Electronics Accessories,Radar Detectors,,,"[uniden, band, undetectability, alert, ultraly...",uniden band undetectability alert ultralyte d...,security & surveillance,C:/Users/sowjanya/Documents/RMTC_IMPLEMENTATIO...,radar detectors


In [ ]:
len(testinp1)+len(testinp2)+len(testinp3)+len(testinp4)+len(testinp5)+len(testinp6)+len(testinp7)+len(testinp8)+len(testinp9)+len(testinp10)+len(testinp11)

2378

In [ ]:
file_paths = [
    'testinp1.csv',  # Replace 'file1.csv' with your actual file paths
    'testinp22.csv','testinp33.csv','testinp5.csv','testinp6.csv','testinp7.csv','testinp9.csv','testinp10.csv','testinp11.csv'

]

# Initialize an empty list to store DataFrames
dfs = []

# Read each CSV file and append its contents to the 'dfs' list
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate all DataFrames in the 'dfs' list into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)  # ignore_index=True resets the indices

# Calculate accuracy between 'leaf' and 'pleaf' columns
accuracy = accuracy_score(combined_df['leaf'], combined_df['pleaf']) * 100
print("Overall accuracy between 'leaf' and 'pleaf' columns:", accuracy)

Overall accuracy between 'leaf' and 'pleaf' columns: 59.88934993084371


In [ ]:
type(dbn_df['leaf'])

Overall accuracy between 'leaf' and 'pleaf' columns: 62.01014292300599

pandas.core.series.Series

In [ ]:
file_path = 'destination.csv'

# Save the DataFrame to a CSV file
dbn_df.to_csv(file_path, index=False)

In [ ]:
def train_denoising_autoencoder(X_train, X_new, X_new_val):
    # Build Denoising Autoencoder (DAE)
    input_dim = X_train.shape[1]
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(512, activation='relu')(input_layer)
    encoded = Dropout(0.5)(encoded)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)

    dae = Model(input_layer, decoded)
    dae.compile(optimizer='adam', loss='binary_crossentropy')
    dae.fit(X_train.toarray(), X_train.toarray(), epochs=40, batch_size=5, validation_data=(X_new_val.toarray(), X_new_val.toarray()))

    # Extract features using DAE
    encoder = Model(input_layer, encoded)
    X_train_encoded = encoder.predict(X_train.toarray())
    X_test_encoded = encoder.predict(X_new.toarray())
    X_val_encoded = encoder.predict(X_new_val.toarray())
    return X_train_encoded, X_test_encoded, X_val_encoded